In [1]:
import geopandas as gpd
import numpy as np
import shapely.geometry
from shapely.geometry import Point
import shapely.ops
import shapely
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# geostl nghbd codes:
    # gravois park = 19
    # benton park west = 30
    # dutchtown = 16
    # mount pleasant = 17

In [2]:
nibrs_df = pd.read_csv('crime_data/nibrs_thru_dec_2023.csv')
nibrs_df

C:\Users\diper\AppData\Local\Temp\ipykernel_4504\382449808.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  nibrs_df = pd.read_csv('crime_data/nibrs_thru_dec_2023.csv')


,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip
0,21000002,1/1/2021,02:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,422,4.0,36,38.629573,-90.204995,1700 CHESTNUT ST,NaN,Saint Louis,MO,NaN
1,21000003,1/1/2021,09:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0
2,21000003,1/1/2021,09:00.0,PROPERTY DAMAGE,A,290,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0
3,21000007,1/1/2021,10:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,223,2.0,9,38.598725,-90.310722,6963 PERNOD AVE,NaN,Saint Louis,MO,63139.0
4,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173359,24000551,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,CITY1,1.0,16,38.587915,-90.252870,3920 DUNNICA AVE,NaN,Saint Louis,MO,NaN
173360,24000557,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,627,6.0,68,38.677708,-90.217160,2007 E FAIR AVE,NaN,Saint Louis,MO,NaN
173361,24000591,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,127,1.0,3,38.569796,-90.254490,5516 GRACE AVE,NaN,Saint Louis,MO,NaN
173362,24000629,12/31/2023,00:00.0,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,A,23G,CITY4,4.0,36,38.630687,-90.201630,1500 OLIVE ST,NaN,Saint Louis,MO,NaN


In [3]:
nibrs_geo = [Point(xy) for xy in zip(nibrs_df['longitude'], nibrs_df['latitude'])]
nibrs_df = gpd.GeoDataFrame(nibrs_df, geometry=nibrs_geo)
nibrs_df['date_inc'] = pd.to_datetime(nibrs_df['date_inc'])

nghbrhd_list = ['16', '17', '19', '30']
mask = nibrs_df['neighborhood'].isin(nghbrhd_list)
nbhd_nibrs_df = nibrs_df[mask]
nbhd_nibrs_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
4,21000008,2021-01-01,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
5,21000008,2021-01-01,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
11,21000032,2021-01-01,52:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,127,1.0,16,38.571658,-90.251776,S GRAND BLVD,LIERMANN AVE,Saint Louis,MO,63116.0,POINT (-90.25178 38.57166)
37,21000075,2021-01-01,37:00.0,RESISTING ARREST/DETENTION/STOP BY FLEEING - C...,B,90Z,322,3.0,16,38.588583,-90.244290,CHIPPEWA ST,S GRAND BLVD,Saint Louis,MO,NaN,POINT (-90.24429 38.58858)
38,21000077,2021-01-01,20:00.0,BURGLARY - 2ND DEGREE,A,220,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173311,23059360,2023-12-31,30:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,127,1.0,16,38.584574,-90.250810,3891 MERAMEC ST,203,Saint Louis,MO,NaN,POINT (-90.25081 38.58457)
173312,23059365,2023-12-31,11:00.0,POSSESSION OF CONTROLLED SUBSTANCE,A,35A,322,3.0,16,38.581622,-90.243918,4130 S. GRAND BOULEVARD,NaN,Saint Louis,MO,63118,POINT (-90.24392 38.58162)
173315,23059375,2023-12-31,45:00.0,ASSAULT 1ST DEGREE OR ATTEMPT,A,13A,323,3.0,19,38.588943,-90.239015,3716 VIRGINIA AVE,NaN,Saint Louis,MO,63118,POINT (-90.23901 38.58894)
173336,24000111,2023-12-31,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,125,1.0,17,38.572195,-90.235080,4625 OREGON AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23508 38.57220)


In [4]:
bpw_gp_df = nbhd_nibrs_df[(nbhd_nibrs_df['neighborhood'] == '19') | (nbhd_nibrs_df['neighborhood'] == '30')].reset_index(drop=True)
bpw_gp_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
0,21000077,2021-01-01,20:00.0,BURGLARY - 2ND DEGREE,A,220,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
1,21000077,2021-01-01,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
2,21015001,2021-01-01,01:00.0,STATUTORY SODOMY OR ATTEMPTED STATUTORY SODOMY...,A,11B,321,3.0,19,38.591852,-90.230130,3500 CALIFORNIA AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23013 38.59185)
3,21038699,2021-01-01,01:00.0,IDENTITY THEFT OR ATTEMPT,A,26F,323,3.0,19,38.589626,-90.238146,3653 S COMPTON AVE,NaN,Saint Louis,MO,63118.0,POINT (-90.23815 38.58963)
4,21055455,2021-01-01,01:00.0,CHILD MOLESTATION - 2ND DEGREE -CHILD LESS THA...,A,11D,325,3.0,30,38.597826,-90.234380,3108 WYOMING ST,NaN,Saint Louis,MO,NaN,POINT (-90.23438 38.59783)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,23059139,2023-12-30,45:00.0,DISCHARGING FIREARM WITHIN CITY (SHOTSPOTTER),A,520,323,3.0,19,38.593012,-90.238444,3300 POTOMAC AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23844 38.59301)
5193,23059217,2023-12-30,50:00.0,TRESPASSING ON PRIVATE PROPERTY,B,90J,321,3.0,19,38.590583,-90.228837,2711 MIAMI,NaN,Saint Louis,MO,63118,POINT (-90.22884 38.59058)
5194,23059231,2023-12-30,49:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,325,3.0,30,38.601067,-90.225990,2916 OHIO AVE,NaN,Saint Louis,MO,63118,POINT (-90.22599 38.60107)
5195,23059308,2023-12-31,01:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,325,3.0,30,38.601758,-90.230450,GRAVOIS AVE,CALIFORNIA,Saint Louis,MO,NaN,POINT (-90.23045 38.60176)


In [5]:
dt_mp_df = nbhd_nibrs_df[(nbhd_nibrs_df['neighborhood'] == '16') | (nbhd_nibrs_df['neighborhood'] == '17')].reset_index(drop=True)
dt_mp_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
0,21000008,2021-01-01,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
1,21000008,2021-01-01,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
2,21000032,2021-01-01,52:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,127,1.0,16,38.571658,-90.251776,S GRAND BLVD,LIERMANN AVE,Saint Louis,MO,63116.0,POINT (-90.25178 38.57166)
3,21000075,2021-01-01,37:00.0,RESISTING ARREST/DETENTION/STOP BY FLEEING - C...,B,90Z,322,3.0,16,38.588583,-90.244290,CHIPPEWA ST,S GRAND BLVD,Saint Louis,MO,NaN,POINT (-90.24429 38.58858)
4,21000112,2021-01-01,30:00.0,TRESPASS,B,90J,127,1.0,16,38.587069,-90.249287,3761 KEOKUK ST,NaN,Saint Louis,MO,63111.0,POINT (-90.24929 38.58707)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11482,23059331,2023-12-31,00:00.0,DOMESTIC ASSAULT - 2ND DEGREE,A,13B,127,1.0,16,38.587053,-90.249017,3747 KEOKUK ST,NaN,Saint Louis,MO,NaN,POINT (-90.24902 38.58705)
11483,23059360,2023-12-31,30:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,127,1.0,16,38.584574,-90.250810,3891 MERAMEC ST,203,Saint Louis,MO,NaN,POINT (-90.25081 38.58457)
11484,23059365,2023-12-31,11:00.0,POSSESSION OF CONTROLLED SUBSTANCE,A,35A,322,3.0,16,38.581622,-90.243918,4130 S. GRAND BOULEVARD,NaN,Saint Louis,MO,63118,POINT (-90.24392 38.58162)
11485,24000111,2023-12-31,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,125,1.0,17,38.572195,-90.235080,4625 OREGON AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23508 38.57220)


In [6]:
gj_shp = gpd.read_file("gj_area.shx")
gj_shp

,geometry
0,"POLYGON ((-90.22300 38.60493, -90.22799 38.581..."


In [7]:
dt_mp_df['geometry'].within(gj_shp.loc[0, 'geometry']).value_counts()

False    7931
True     3556
Name: count, dtype: int64

In [8]:
geo_mask = dt_mp_df['geometry'].within(gj_shp.loc[0, 'geometry'])
dt_mp_df = dt_mp_df[geo_mask].reset_index(drop=True)

### Here is where i have both dfs and need to combine by month and get the crime counts

In [ ]:
nibrs_codes_dict = {'Person': ['09A', '09B', '09C', '09D', '100', '11A', '11B', '11C', '11D', '13A', '13A*', '13B', '13C', '36A', '36B', '64A', '64B'], 
                    'Property': ['120', '120*', '200', '210', '220', '23A', '23B', '23C', '23D', '23E', '23F', '23G', '23H', '240', '250', '26A', '26B', '26C', '26D', '26E', '26F', '26G', '270', '280', '290', '510', '90A'],
                    'Society': ['35A', '35B', '370', '39A', '39B', '39C', '39D', '40A', '40B', '40C', '520', '720', '90B', '90C', '90D', '90E', '90F', '90G', '90H', '90J'],
                    'Unspecified': ['90Z']}

# how to classify violent/nonviolent crimes?

nibrs_col_list = nibrs_codes_dict['Person'] + ['Person'] + nibrs_codes_dict['Property'] + ['Property'] + nibrs_codes_dict['Society'] + ['Society'] + nibrs_codes_dict['Unspecified'] + ['Unspecified'] + ['Total']
nibrs_col_dict = {i:0 for i in nibrs_col_list}

In [148]:
gj_df = pd.concat([dt_mp_df, bpw_gp_df], ignore_index=True)
gj_df = gj_df.groupby([pd.Grouper(key='date_inc', freq='M'),dt_mp_df['nibrs_code']])['inci_id'].count()

gj_df = gj_df.unstack().fillna(0)
# rename(columns= nibrs_col_list)
gj_df.columns.name = None

missing_cols = [col for col in nibrs_col_list if col not in gj_df.columns]
gj_df[missing_cols] = 0

gj_df = gj_df[nibrs_col_list]

gj_df.loc[:,'Person'] = gj_df.loc[:,'09A':'64B'].sum(axis=1)

gj_df.loc[:,'Property'] = gj_df.loc[:,'120':'90A'].sum(axis=1)
gj_df.loc[:,'Society'] = gj_df.loc[:,'35A':'90J'].sum(axis=1)
gj_df.loc[:,'Unspecified'] = gj_df.loc[:,'90Z']
gj_df.loc[:,'Total'] = gj_df.loc[:,['Person', 'Property', 'Society', 'Unspecified']].sum(axis=1)


gj_df = gj_df.astype('int64')
gj_df

,09A,09B,09C,09D,100,11A,11B,11C,11D,13A,13A*,13B,13C,36A,36B,64A,64B,Person,120,120*,200,210,220,23A,23B,23C,23D,23E,23F,23G,23H,240,250,26A,26B,26C,26D,26E,26F,26G,270,280,290,510,90A,Property,35A,35B,370,39A,39B,39C,39D,40A,40B,40C,520,720,90B,90C,90D,90E,90F,90G,90H,90J,Society,90Z,Unspecified,Total
date_inc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-31,1,0,0,0,0,0,1,0,1,5,0,3,0,0,0,0,0,11,2,0,0,0,3,0,0,1,1,0,6,5,2,5,0,0,0,0,0,0,0,0,0,0,6,0,0,31,8,3,0,0,0,0,0,0,0,0,6,0,0,1,0,0,0,0,0,0,18,20,20,80
2021-02-28,2,0,0,0,0,0,1,0,0,3,0,3,0,0,1,0,0,10,1,0,0,0,2,0,1,0,3,0,5,3,0,4,1,0,0,0,0,0,0,0,0,0,5,0,0,25,13,4,0,0,0,0,0,0,0,0,5,0,0,2,1,0,0,0,0,1,26,9,9,70
2021-03-31,0,0,0,0,0,0,0,0,0,6,0,3,2,0,0,0,0,11,1,0,0,0,2,0,0,0,5,0,4,3,2,8,1,0,0,0,0,0,0,0,0,3,19,0,0,48,17,3,0,0,0,0,0,0,0,0,15,0,1,3,0,0,0,0,0,1,40,22,22,121
2021-04-30,1,0,0,0,1,1,0,0,0,5,0,5,0,0,0,0,0,13,1,0,0,0,5,0,1,1,2,0,3,4,4,9,0,0,1,0,0,0,0,0,0,0,19,0,0,50,11,2,0,0,0,0,0,0,0,0,7,0,2,1,0,0,0,0,0,0,23,17,17,103
2021-05-31,1,0,0,0,0,1,2,0,0,4,0,9,3,0,0,0,0,20,2,0,0,0,3,0,0,0,4,0,2,2,5,16,0,0,1,0,0,0,0,0,0,0,11,0,0,46,5,2,0,0,0,0,0,0,0,0,6,0,0,3,0,0,1,0,0,1,18,19,19,103
2021-06-30,0,0,0,0,0,0,0,0,1,11,0,13,1,0,0,0,0,26,1,0,1,0,4,0,0,0,2,0,1,2,5,5,0,0,0,0,0,0,0,0,0,0,11,0,0,32,5,3,0,0,0,0,0,0,0,0,14,0,0,5,0,0,0,0,0,2,29,10,10,97
2021-07-31,0,0,0,0,0,0,0,0,0,6,0,4,2,0,0,0,0,12,1,0,0,0,2,0,0,0,2,0,7,6,2,9,0,0,0,0,0,0,0,0,0,2,16,0,0,47,6,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,1,17,10,10,86
2021-08-31,0,0,0,0,0,0,0,0,0,6,0,9,2,0,0,0,0,17,2,0,1,0,6,0,0,0,2,0,5,5,7,4,1,0,1,0,0,0,0,0,0,4,13,0,0,51,5,3,0,0,0,0,0,0,0,0,7,0,0,3,0,0,0,0,0,3,21,21,21,110
2021-09-30,1,0,0,0,0,1,1,0,0,6,0,3,1,0,0,0,0,13,5,0,0,0,5,0,0,0,3,0,9,9,5,10,0,0,0,0,0,0,0,0,0,7,21,0,0,74,9,3,0,0,0,0,0,0,0,0,5,0,0,1,0,0,1,0,0,4,23,15,15,125


In [ ]:
gj_df.to_csv('gravois-jefferson_nibrs_data_2021-2023')

In [34]:
# crime_counts_df = dt_mp_df.groupby([pd.Grouper(key='date_inc', freq='M'),dt_mp_df['nibrs_code']])['inci_id'].count()
# crime_counts_df

date_inc    nibrs_code
2021-01-31  09A            1
            11B            1
            11D            1
            120            2
            13A            5
                          ..
2023-12-31  35A            7
            35B            2
            520            4
            90C            1
            90Z           16
Name: inci_id, Length: 727, dtype: int64

In [46]:
# str(crime_counts_df.index[0][0])

'2021-01-31 00:00:00'

In [53]:
# for idx in crime_counts_df.index:
#     month = str(idx[0]).strip(' 00:00:00')
# print(month)

2023-12-31


In [141]:
# test_df = gj_df
# test_df = test_df.unstack().fillna(0)
# # rename(columns= nibrs_col_list)
# test_df.columns.name = None

# missing_cols = [col for col in nibrs_col_list if col not in test_df.columns]
# test_df[missing_cols] = 0

# test_df = test_df[nibrs_col_list]
# test_df

# crime_agg_df = test_df

# crime_agg_df.loc[:,'Person'] = crime_agg_df.loc[:,'09A':'64B'].sum(axis=1)

# crime_agg_df.loc[:,'Property'] = crime_agg_df.loc[:,'120':'90A'].sum(axis=1)
# crime_agg_df.loc[:,'Society'] = crime_agg_df.loc[:,'35A':'90J'].sum(axis=1)
# crime_agg_df.loc[:,'Unspecified'] = crime_agg_df.loc[:,'90Z']
# crime_agg_df.loc[:,'Total'] = crime_agg_df.loc[:,['Person', 'Property', 'Society', 'Unspecified']].sum(axis=1)

# crime_agg_df

,09A,09B,09C,09D,100,11A,11B,11C,11D,13A,13A*,13B,13C,36A,36B,64A,64B,Person,120,120*,200,210,220,23A,23B,23C,23D,23E,23F,23G,23H,240,250,26A,26B,26C,26D,26E,26F,26G,270,280,290,510,90A,Property,35A,35B,370,39A,39B,39C,39D,40A,40B,40C,520,720,90B,90C,90D,90E,90F,90G,90H,90J,Society,90Z,Unspecified,Total
date_inc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-31,1.0,0.0,0,0,0.0,0.0,1.0,0,1.0,5.0,0,3.0,0.0,0,0.0,0,0,11,2.0,0,0.0,0,3.0,0.0,0.0,1.0,1.0,0,6.0,5.0,2.0,5.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,6.0,0,0,31,8.0,3.0,0.0,0,0,0,0,0,0,0,6.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0.0,18,20.0,20,80
2021-02-28,2.0,0.0,0,0,0.0,0.0,1.0,0,0.0,3.0,0,3.0,0.0,0,1.0,0,0,10,1.0,0,0.0,0,2.0,0.0,1.0,0.0,3.0,0,5.0,3.0,0.0,4.0,1.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,5.0,0,0,25,13.0,4.0,0.0,0,0,0,0,0,0,0,5.0,0.0,0.0,2.0,1.0,0,0.0,0.0,0,1.0,26,9.0,9,70
2021-03-31,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,6.0,0,3.0,2.0,0,0.0,0,0,11,1.0,0,0.0,0,2.0,0.0,0.0,0.0,5.0,0,4.0,3.0,2.0,8.0,1.0,0.0,0.0,0,0,0,0.0,0,0.0,3.0,19.0,0,0,48,17.0,3.0,0.0,0,0,0,0,0,0,0,15.0,0.0,1.0,3.0,0.0,0,0.0,0.0,0,1.0,40,22.0,22,121
2021-04-30,1.0,0.0,0,0,1.0,1.0,0.0,0,0.0,5.0,0,5.0,0.0,0,0.0,0,0,13,1.0,0,0.0,0,5.0,0.0,1.0,1.0,2.0,0,3.0,4.0,4.0,9.0,0.0,0.0,1.0,0,0,0,0.0,0,0.0,0.0,19.0,0,0,50,11.0,2.0,0.0,0,0,0,0,0,0,0,7.0,0.0,2.0,1.0,0.0,0,0.0,0.0,0,0.0,23,17.0,17,103
2021-05-31,1.0,0.0,0,0,0.0,1.0,2.0,0,0.0,4.0,0,9.0,3.0,0,0.0,0,0,20,2.0,0,0.0,0,3.0,0.0,0.0,0.0,4.0,0,2.0,2.0,5.0,16.0,0.0,0.0,1.0,0,0,0,0.0,0,0.0,0.0,11.0,0,0,46,5.0,2.0,0.0,0,0,0,0,0,0,0,6.0,0.0,0.0,3.0,0.0,0,1.0,0.0,0,1.0,18,19.0,19,103
2021-06-30,0.0,0.0,0,0,0.0,0.0,0.0,0,1.0,11.0,0,13.0,1.0,0,0.0,0,0,26,1.0,0,1.0,0,4.0,0.0,0.0,0.0,2.0,0,1.0,2.0,5.0,5.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,11.0,0,0,32,5.0,3.0,0.0,0,0,0,0,0,0,0,14.0,0.0,0.0,5.0,0.0,0,0.0,0.0,0,2.0,29,10.0,10,97
2021-07-31,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,6.0,0,4.0,2.0,0,0.0,0,0,12,1.0,0,0.0,0,2.0,0.0,0.0,0.0,2.0,0,7.0,6.0,2.0,9.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,2.0,16.0,0,0,47,6.0,0.0,0.0,0,0,0,0,0,0,0,10.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,1.0,17,10.0,10,86
2021-08-31,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,6.0,0,9.0,2.0,0,0.0,0,0,17,2.0,0,1.0,0,6.0,0.0,0.0,0.0,2.0,0,5.0,5.0,7.0,4.0,1.0,0.0,1.0,0,0,0,0.0,0,0.0,4.0,13.0,0,0,51,5.0,3.0,0.0,0,0,0,0,0,0,0,7.0,0.0,0.0,3.0,0.0,0,0.0,0.0,0,3.0,21,21.0,21,110
2021-09-30,1.0,0.0,0,0,0.0,1.0,1.0,0,0.0,6.0,0,3.0,1.0,0,0.0,0,0,13,5.0,0,0.0,0,5.0,0.0,0.0,0.0,3.0,0,9.0,9.0,5.0,10.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,7.0,21.0,0,0,74,9.0,3.0,0.0,0,0,0,0,0,0,0,5.0,0.0,0.0,1.0,0.0,0,1.0,0.0,0,4.0,23,15.0,15,125


In [54]:
# dec_dt_mp = dt_mp_df[dt_mp_df['date_inc'].str.contains('12/.*/2023', regex=True)]
# dec_bpw_gp = bpw_gp_df[bpw_gp_df['date_inc'].str.contains('12/.*/2023', regex=True)]

# dec = pd.concat([dec_dt_mp, dec_bpw_gp], ignore_index=True)

In [ ]:

# crime_agg_df.loc[curr_idx, 'Person'] = crime_agg_df.loc[curr_idx,'09A':'64B'].sum()
# crime_agg_df.loc[curr_idx, 'Property'] = crime_agg_df.loc[curr_idx,'120':'90A'].sum()
# crime_agg_df.loc[curr_idx, 'Society'] = crime_agg_df.loc[curr_idx,'35A':'90J'].sum()
# crime_agg_df.loc[curr_idx, 'Unspecified'] = crime_agg_df.loc[curr_idx,'90Z'].sum()
# crime_agg_df.loc[curr_idx, 'Total'] = crime_agg_df.loc[curr_idx,['Person', 'Property', 'Society', 'Unspecified']].sum()